In [4]:
import os

import numpy as np
import torch
from lib.config import cfg

from lib.dataset.dataset2D import Dataset2D
from lib.dataset.dataset3D import Dataset3D
from lib.hybridnet.modules.efficienttrack.efficienttrack import EfficientTrack
import lib.hybridnet.modules.efficienttrack.darkpose as darkpose

from lib.config.project_manager import ProjectManager

project = ProjectManager()
project.load('Ralph_Center_Test')

#project.create_new(
#    name = 'CSV Creation',
#    dataset2D_path = '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen',
#    dataset3D_path = '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen')

cfg = project.get_cfg()

Successfully loaded project Ralph_Center_Test!


In [7]:
training_set = Dataset3D(cfg, set='train')
val_set = Dataset3D(cfg, set='val')
calibPaths = [training_set.coco.dataset['calibration']['intrinsics'], 
              training_set.coco.dataset['calibration']['extrinsics']]


loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [6]:
# import lib.hybridnet.modules.efficienttrack.darkpose as darkpose
import cv2
import matplotlib.pyplot as plt

weights = "/home/timo/Desktop/HybridNet/projects/Ralph_Center_Test/models/efficienttrack/EfficientTrack-d0_210.pth"

efficientTrack = EfficientTrack('inference', cfg, weights)

In [ ]:
import cv2
import csv
import matplotlib.pyplot as plt
import itertools
from joblib import Parallel, delayed

    

video_paths = [
    '/media/timo/Elements/Example_Recording/Camera_B.mp4',
    '/media/timo/Elements/Example_Recording/Camera_LBB.mp4',
    '/media/timo/Elements/Example_Recording/Camera_LBT.mp4',
    '/media/timo/Elements/Example_Recording/Camera_LC.mp4',
    '/media/timo/Elements/Example_Recording/Camera_LFB.mp4',
    '/media/timo/Elements/Example_Recording/Camera_LFT.mp4',
    '/media/timo/Elements/Example_Recording/Camera_RBB.mp4',
    '/media/timo/Elements/Example_Recording/Camera_RBT.mp4',
    '/media/timo/Elements/Example_Recording/Camera_RC.mp4',
    '/media/timo/Elements/Example_Recording/Camera_RFB.mp4',
    '/media/timo/Elements/Example_Recording/Camera_RFT.mp4',
    '/media/timo/Elements/Example_Recording/Camera_T.mp4', 
]

caps = []
outs = []
for path in video_paths:
    caps.append(cv2.VideoCapture(path))
    outs.append(cv2.VideoWriter(os.path.join('/home/timo/Desktop/HybridNet/Videos_Ralph_Center', path.split('/')[-1].split(".")[0] + ".avi"), cv2.VideoWriter_fourcc('M','J', 'P', 'G'), 100, (1280,1024)))

for cap in caps:
        cap.set(1,0) #99640
        
def process(cap):
    ret, img = cap.read()
    return img
        
counter = 0
    
ret = True
while ret:
    if counter % 20 == 0:
        print (counter)
    counter += 1
    imgs_orig = []
    centerHMs = []
    camsToUse = []

    imgs_orig = Parallel(n_jobs=12, require='sharedmem')(delayed(process)(cap) for cap in caps)
    imgs = torch.zeros(12,3,256,320)
    for i,img in enumerate(imgs_orig[:]):
        img = ((cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)/255.0-cfg.DATASET.MEAN)/cfg.DATASET.STD)
        img = cv2.resize(img, (320,256))
        imgs[i] = torch.from_numpy(cv2.resize(img, (320,256)).transpose(2,0,1))
    
    imgs = imgs.cuda()
    outputs = efficientTrack.model(imgs)
    preds, maxvals = darkpose.get_final_preds(outputs[1].clamp(0,255).detach().cpu().numpy(), None)
    camsToUse = []
    
    for i,val in enumerate(maxvals[:]):
        if val > 180:
            camsToUse.append(i)
    
    if len(camsToUse) >= 2:
        center3D = torch.from_numpy(val_set.reproTool.reconstructPoint((preds.reshape(12,2)*8).transpose(), camsToUse))
        reproPoints = val_set.reproTool.reprojectPoint(center3D)
        
        errors = []
        errors_valid = []
        for i in range(12):
            if maxvals[i] > 180:
                errors.append(np.linalg.norm(preds.reshape(12,2)[i]*8-reproPoints[i]))
                errors_valid.append(np.linalg.norm(preds.reshape(12,2)[i]*8-reproPoints[i]))
            else:
                errors.append(0)
        medianError = np.median(np.array(errors_valid))
        print ("Error: ", medianError)
        print ("Var based: ", medianError+2*np.sqrt(np.var(errors_valid)),medianError*4)
        camsToUse = []
        for i,val in enumerate(maxvals[:]):
            if val > 180 and errors[i] < 2*medianError:
                camsToUse.append(i)
        center3D = torch.from_numpy(val_set.reproTool.reconstructPoint((preds.reshape(12,2)*8).transpose(), camsToUse))
        reproPoints = val_set.reproTool.reprojectPoint(center3D)

        for i in range(12):
            colors = [(255,0,0), (255,0,0),(255,0,0),(255,0,0),(0,255,0),(0,255,0),(0,255,0),(0,255,0),(0,0,255),(0,0,255),(0,0,255),(0,0,255),(255,255,0),(255,255,0),(255,255,0), (255,255,0),
                              (0,255,255),(0,255,255),(0,255,255),(0,255,255), (255,0,255),(100,0,100),(100,0,100)]

            cv2.circle(imgs_orig[i], (int(reproPoints[i][0]), int(reproPoints[i][1])), 2, (255,100,100), thickness=5)
            for j in range(12):
                if j in camsToUse:
                    cv2.circle(imgs_orig[i], (100+j*90,200), int(errors[j]), (255,0,0), thickness=5)
                else:
                    cv2.circle(imgs_orig[i], (100+j*90,200), int(5), (100,100,100), thickness=5)
            #cv2.putText(imgs_orig[i],str(error/12),(10,500), font, 4, (255, 0, 0), 2, cv2.LINE_AA)
            heatmap = outputs[1][i].clamp(0,255).detach().transpose(2,0).transpose(0,1).cpu().int().numpy().astype(np.uint8)
            heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
            added_image = cv2.addWeighted(imgs_orig[i],1.0, cv2.resize(heatmap, (1280,1024)),0.25,0)
            outs[i].write(added_image)
    

    #hetamap = cv2.resize(outputs[1].clamp(0,255).detach().cpu().numpy()[11][0]/255., (320,256), interpolation=cv2.cv2.INTER_NEAREST)
    #plt.imshow(hetamap, alpha=0.6)
    #plt.show()
    

0
Error:  4.043244804643754
Var based:  8.458593977335024 16.172979218575016
Error:  4.552894069207638
Var based:  8.652528757821262 18.211576276830552
Error:  4.411844995543832
Var based:  8.504866652076938 17.647379982175327
Error:  4.479617458126379
Var based:  8.518594662475081 17.918469832505515
Error:  4.321210284888928
Var based:  8.43048675642892 17.284841139555713
Error:  4.366236195224043
Var based:  8.282400584822522 17.46494478089617
Error:  4.45872202498241
Var based:  8.369149967901443 17.83488809992964
Error:  4.466352318122926
Var based:  8.378259343104727 17.865409272491704
Error:  4.473053984355751
Var based:  8.360437966750869 17.892215937423003
Error:  4.802625462065858
Var based:  8.733584055778588 19.210501848263434
Error:  4.721016364088294
Var based:  8.625971483927273 18.884065456353177
Error:  4.482176757801329
Var based:  8.36226479489946 17.928707031205317
Error:  4.4583083157603784
Var based:  8.337519264648854 17.833233263041514
Error:  4.547312641552649
V

Error:  5.507963732114009
Var based:  9.06398065800367 22.031854928456035
Error:  5.509177592788877
Var based:  9.04004222888563 22.036710371155507
Error:  5.545772800537592
Var based:  9.123281740151974 22.18309120215037
Error:  5.569670847213149
Var based:  9.009643479553617 22.278683388852595
Error:  5.582540862832487
Var based:  9.02771652461816 22.33016345132995
Error:  5.556090332128302
Var based:  8.944102915868696 22.224361328513208
Error:  5.427480898596224
Var based:  8.833722926448091 21.709923594384897
Error:  5.44060815985438
Var based:  8.725689921403527 21.76243263941752
Error:  5.46476897320211
Var based:  8.652430669039752 21.85907589280844
Error:  5.619848576640258
Var based:  10.024189128749194 22.479394306561034
120
Error:  5.699447850053555
Var based:  10.146377525522226 22.79779140021422
Error:  5.6860734625080855
Var based:  10.161713035658638 22.744293850032342
Error:  5.7441877764743765
Var based:  10.19611417173246 22.976751105897506
Error:  5.609748906969731


Error:  3.2019502977374694
Var based:  6.99698174638023 12.807801190949878
Error:  3.2218367528920737
Var based:  6.988289942923306 12.887347011568295
Error:  3.364944061985177
Var based:  7.122131129972055 13.459776247940708
Error:  3.1732733542302483
Var based:  7.047169127766019 12.693093416920993
Error:  3.1004843420787243
Var based:  7.023296992392536 12.401937368314897
Error:  3.163206744823595
Var based:  7.004814553552425 12.65282697929438
Error:  3.105382621825241
Var based:  6.9905460162422335 12.421530487300965
Error:  3.1663539233832374
Var based:  6.981213439186949 12.66541569353295
Error:  3.0673262256964433
Var based:  6.970020790785038 12.269304902785773
Error:  3.4890845988045327
Var based:  7.2285490104773125 13.95633839521813
Error:  3.7381582090754093
Var based:  7.403448889352212 14.952632836301637
Error:  3.9086947949496906
Var based:  7.499365387189645 15.634779179798763
Error:  3.908247146026449
Var based:  7.473574902789183 15.632988584105796
Error:  3.59933246

Error:  4.2491240045720975
Var based:  9.370545560940549 16.99649601828839
Error:  4.191030874599571
Var based:  9.398051720640506 16.764123498398284
Error:  4.189327656251329
Var based:  9.394940669561738 16.757310625005317
Error:  4.171459809157563
Var based:  9.379074074429582 16.68583923663025
Error:  4.341869354238124
Var based:  9.448702149372465 17.367477416952497
Error:  4.329621083556087
Var based:  9.592505522288516 17.318484334224348
Error:  4.405191886838921
Var based:  9.621542100900594 17.620767547355683
Error:  4.3750843152918115
Var based:  9.621543259351313 17.500337261167246
Error:  4.475220385676004
Var based:  9.674394583429452 17.900881542704017
Error:  4.485524616476614
Var based:  9.700337620336764 17.942098465906454
340
Error:  4.521214251481826
Var based:  9.724308547984968 18.084857005927304
Error:  4.495790140928839
Var based:  9.72280037147624 17.983160563715355
Error:  4.512702330978854
Var based:  9.721539697865705 18.050809323915416
Error:  4.539335611425

Error:  3.78980454748532
Var based:  9.87146915804103 15.15921818994128
Error:  3.778887239843723
Var based:  9.843351116881605 15.115548959374893
Error:  3.559819455040815
Var based:  9.661054472595113 14.23927782016326
Error:  3.5105251574198557
Var based:  9.6138440019512 14.042100629679423
Error:  3.58175511670183
Var based:  9.586551274774635 14.32702046680732
Error:  3.5001288444528513
Var based:  9.522138862643724 14.000515377811405
Error:  3.4347559208147835
Var based:  9.411238710815248 13.739023683259134
Error:  3.3357796376442437
Var based:  9.321131090093449 13.343118550576975
Error:  3.340829539671618
Var based:  9.2719742076623 13.363318158686472
Error:  3.2852365288567182
Var based:  9.192510263633581 13.140946115426873
Error:  3.2936941424371224
Var based:  9.169616319240191 13.17477656974849
Error:  3.3974271071547797
Var based:  9.274995948666996 13.589708428619119
Error:  3.6186349163884755
Var based:  9.530692820887452 14.474539665553902
Error:  3.699936842344665
Va

...

Error:  7.459334496611882
Var based:  14.701239455057355 29.83733798644753
2300
Error:  8.254148422430678
Var based:  14.841088010826473 33.01659368972271
Error:  7.898129922949417
Var based:  15.854886678215287 31.59251969179767
Error:  8.927048176087022
Var based:  15.676488235175237 35.708192704348086
Error:  7.5472155157917
Var based:  15.85277573794514 30.1888620631668
Error:  5.4843278784128415
Var based:  15.188513254877247 21.937311513651366
Error:  5.356625579760274
Var based:  14.678921566762344 21.426502319041095
Error:  5.534023324924792
Var based:  15.534063554471448 22.13609329969917
Error:  5.87568860963783
Var based:  13.867713848686126 23.50275443855132
Error:  5.824334813055643
Var based:  13.47327804163681 23.29733925222257
Error:  5.6404801312054955
Var based:  12.871942068965222 22.561920524821982
Error:  6.219335172529559
Var based:  13.474694676532188 24.877340690118235
Error:  6.248640418221456
Var based:  12.676879230395365 24.994561672885823
Error:  5.78328923

Error:  5.982354872802856
Var based:  8.5666428540464 23.929419491211423
Error:  5.600344322229798
Var based:  7.827953987146082 22.401377288919193
Error:  5.549742393936694
Var based:  10.043480321339107 22.198969575746776
Error:  5.179310080353313
Var based:  9.875826332579086 20.717240321413254
Error:  5.47962578290892
Var based:  9.852839979503344 21.91850313163568
Error:  5.629601016633124
Var based:  9.959091663963914 22.518404066532497
Error:  5.603877721904056
Var based:  9.628395060027302 22.415510887616225
Error:  5.838071087060268
Var based:  10.127123500028706 23.35228434824107
Error:  5.759391737442144
Var based:  10.204593781522497 23.037566949768575
Error:  5.816710085219571
Var based:  10.408111230644154 23.266840340878282
Error:  6.0366959549247206
Var based:  10.518125991987045 24.146783819698882
Error:  6.015688423288307
Var based:  10.286764243559762 24.06275369315323
2420
Error:  6.457625700145212
Var based:  10.349496079828235 25.83050280058085
Error:  7.071914415

Error:  6.603896577021729
Var based:  13.483584187838266 26.415586308086915
Error:  7.0974035948635725
Var based:  15.751547918580101 28.38961437945429
Error:  6.749606573376452
Var based:  9.782227652388716 26.99842629350581
2520
Error:  7.110576764611634
Var based:  11.543231386747067 28.442307058446534
Error:  5.741723291486894
Var based:  13.034446068782813 22.966893165947575
Error:  6.07708640599712
Var based:  13.62773340558173 24.30834562398848
Error:  5.7716132566853595
Var based:  11.206001089748622 23.086453026741438
Error:  5.15908440792641
Var based:  10.747785575000934 20.63633763170564
Error:  5.049027679703356
Var based:  10.634201004421216 20.196110718813426
Error:  6.53869882086414
Var based:  12.180972134071126 26.15479528345656
Error:  6.049516571042325
Var based:  11.172355777856078 24.1980662841693
Error:  5.7005908401993155
Var based:  9.171415383002937 22.802363360797262
Error:  5.241891337859977
Var based:  8.789095486951222 20.96756535143991
Error:  4.972597679

Error:  3.267287033469356
Var based:  6.450835246479751 13.069148133877423
Error:  3.2635413230948744
Var based:  6.470539512327874 13.054165292379498
Error:  3.3243997768828453
Var based:  6.557053029466461 13.297599107531381
Error:  3.3301740528695962
Var based:  6.718951905672821 13.320696211478385
Error:  3.3332706179929645
Var based:  6.662410642636598 13.333082471971858
Error:  3.289254949880881
Var based:  6.61382978534468 13.157019799523525
Error:  3.2397082246869204
Var based:  6.28075476730887 12.958832898747682
Error:  3.2541711076064788
Var based:  6.3364063889168865 13.016684430425915
Error:  3.3128184088902977
Var based:  6.381638313070178 13.25127363556119
Error:  3.3715797343379643
Var based:  6.4849076985971985 13.486318937351857
Error:  3.354527675699225
Var based:  6.51163251847054 13.4181107027969
Error:  3.4271639428505813
Var based:  6.1642259111142454 13.708655771402325
Error:  3.23266507410718
Var based:  5.807551349159134 12.93066029642872
Error:  3.30553118916

Error:  3.2391485050792705
Var based:  5.553605687281548 12.956594020317082
Error:  3.139793295730514
Var based:  5.571407257813089 12.559173182922056
Error:  3.1174361241037634
Var based:  5.595904746166623 12.469744496415053
Error:  3.757681869827385
Var based:  6.395855203019834 15.03072747930954
Error:  3.8270324679622507
Var based:  6.472857818421282 15.308129871849003
2740
Error:  3.923959783133628
Var based:  6.677947257911073 15.695839132534513
Error:  3.8432828523422318
Var based:  6.320367426725314 15.373131409368927
Error:  3.93886006295911
Var based:  6.370518746771764 15.75544025183644
Error:  3.799487440626951
Var based:  6.197630502060749 15.197949762507804
Error:  4.02144425307102
Var based:  6.536885535376373 16.08577701228408
Error:  4.031861011711958
Var based:  6.533978976891527 16.12744404684783
Error:  3.885847746343065
Var based:  6.293229798988623 15.54339098537226
Error:  3.9596416280488764
Var based:  6.460640042188032 15.838566512195506
Error:  3.695118215265

In [ ]:
import cv2
import csv
import matplotlib.pyplot as plt
import itertools
from joblib import Parallel, delayed

    

video_paths = [
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_B.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_LBB.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_LBT.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_LC.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_LFB.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_LFT.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_RBB.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_RBT.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_RC.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_RFB.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_RFT.avi',
    '//media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_T.avi', 
]

caps = []
outs = []
for path in video_paths:
    caps.append(cv2.VideoCapture(path))
    outs.append(cv2.VideoWriter(os.path.join('/home/timo/Desktop/HybridNet/Videos_Ralph_Center', path.split('/')[-1]), cv2.VideoWriter_fourcc('M','J', 'P', 'G'), 100, (1280,1024)))

for cap in caps:
        cap.set(1,0) #99640
        
def process(cap):
    ret, img = cap.read()
    return img
        
counter = 0
    
ret = True
while ret:
    if counter % 20 == 0:
        print (counter)
    counter += 1
    imgs_orig = []
    centerHMs = []
    camsToUse = []

    imgs_orig = Parallel(n_jobs=12, require='sharedmem')(delayed(process)(cap) for cap in caps)
    imgs = torch.zeros(12,3,256,320)
    for i,img in enumerate(imgs_orig[:]):
        img = ((cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)/255.0-cfg.DATASET.MEAN)/cfg.DATASET.STD)
        img = cv2.resize(img, (320,256))
        imgs[i] = torch.from_numpy(cv2.resize(img, (320,256)).transpose(2,0,1))
    
    imgs = imgs.cuda()
    outputs = efficientTrack.model(imgs)
    preds, maxvals = darkpose.get_final_preds(outputs[1].clamp(0,255).detach().cpu().numpy(), None)
    camsToUse = []
    print (maxvals[0])
    for i,val in enumerate(maxvals):
        if val > 200:
            camsToUse.append(i)
    
    print (preds.shape)
    
    print ("Error: ", error/12)
    
    if len(camsToUse) >= 2:
        center3D = torch.from_numpy(val_set.reproTool.reconstructPoint((preds.reshape(12,2)*8).transpose(), camsToUse))
        center3D = center3D.int()
        reproPoints = val_set.reproTool.reprojectPoint(center3D)
        error = 0
        for i in range(12):
            if maxvals[i] > 200:
                error += np.linalg.norm(preds.reshape(12,2)[i]*8-reproPoints[i])
        print (reproPoints[0])
        for i in range(12):
            colors = [(255,0,0), (255,0,0),(255,0,0),(255,0,0),(0,255,0),(0,255,0),(0,255,0),(0,255,0),(0,0,255),(0,0,255),(0,0,255),(0,0,255),(255,255,0),(255,255,0),(255,255,0), (255,255,0),
                              (0,255,255),(0,255,255),(0,255,255),(0,255,255), (255,0,255),(100,0,100),(100,0,100)]

            #cv2.circle(imgs_orig[i], (int(reproPoints[i][0]), int(reproPoints[i][1])), 2, (255,100,100), thickness=5)
            #cv2.circle(imgs_orig[i], (200,200), int(error/12), (255,0,0), thickness=5)#
            heatmap = outputs[1][i].clamp(0,255).detach().transpose(2,0).transpose(0,1).cpu().int().numpy().astype(np.uint8)
            heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
            added_image = cv2.addWeighted(imgs_orig[i],1.0, cv2.resize(heatmap, (1280,1024)),0.3,0)
            #cv2.putText(imgs_orig[i],str(error/12),(10,500), font, 4, (255, 0, 0), 2, cv2.LINE_AA)
            outs[i].write(added_image)
            font = cv2.FONT_HERSHEY_SIMPLEX
    

    #hetamap = cv2.resize(outputs[1].clamp(0,255).detach().cpu().numpy()[11][0]/255., (320,256), interpolation=cv2.cv2.INTER_NEAREST)
    #plt.imshow(hetamap, alpha=0.6)
    #plt.show()
    

In [ ]:
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import cm

figure = plt.figure()
axes = figure.gca(projection='3d')

c = ['r', 'r','r','r','b','b','b','b','g','g','g','g', 'orange', 'orange','orange','orange', 'y','y','y','y','purple', 'purple','purple']
line_idxs = [[0,1], [1,2], [2,3], [4,5], [5,6], [6,7], [8,9], [9,10], [10,11], [12,13], [13,14], [14,15], [16,17], [17,18], [18,19], [3,7], [7,11], [11,15], [3,21], [7,21],[11,22], [15,22],[21,22], [18,15], [19,22]]
points3D = points3D_net[0].cpu().numpy()
center3D = center3D.cpu()
for i, point in enumerate(points3D):
    #print ("Classic:", i, point)
    if i != 20:
        axes.scatter(point[0], point[1], point[2], color = c[i])
for line in line_idxs:
    axes.plot([points3D[line[0]][0], points3D[line[1]][0]], [points3D[line[0]][1], points3D[line[1]][1]], [points3D[line[0]][2], points3D[line[1]][2]], c = 'gray')

axes.set_xlim3d(center3D[0]-100, center3D[0]+100)
axes.set_ylim3d(center3D[1]-100, center3D[1]+100)
axes.set_zlim3d(center3D[2]-100, center3D[2]+100)
plt.subplots_adjust(left=0., right=1., top=1., bottom=0.)
#plt.gca().invert_zaxis()
plt.savefig('test.png')
plt.show()